In [ ]:
#THESE SCRIPTS TAKE US FROM RAW DATA TO THE TABLES WE HAVE IN OUR SCHEMA - SEE movie_db_model.png

In [1]:
import pandas as pd
import numpy as np
from itertools import chain

In [2]:
##BEGIN CREATION OF DIRECTOR_LOOKUP AND WRITER_LOOKUP TABLES
df_name_basics = pd.read_csv('raw_data/name_basics.tsv', sep='\t')
print(list(df_name_basics))
##Bring in titles
df_title = pd.read_csv('app_data/title.csv')
title_ids = pd.Series(df_title['title_id'])


['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles']


In [3]:
print(df_name_basics[:10])

      nconst      primaryName birthYear deathYear  \
0  nm0000001     Fred Astaire      1899      1987   
1  nm0000002    Lauren Bacall      1924      2014   
2  nm0000003  Brigitte Bardot      1934        \N   
3  nm0000004     John Belushi      1949      1982   
4  nm0000005   Ingmar Bergman      1918      2007   
5  nm0000006   Ingrid Bergman      1915      1982   
6  nm0000007  Humphrey Bogart      1899      1957   
7  nm0000008    Marlon Brando      1924      2004   
8  nm0000009   Richard Burton      1925      1984   
9  nm0000010     James Cagney      1899      1986   

                primaryProfession                           knownForTitles  
0  soundtrack,actor,miscellaneous  tt0072308,tt0050419,tt0053137,tt0043044  
1              actress,soundtrack  tt0037382,tt0071877,tt0117057,tt0038355  
2     actress,soundtrack,producer  tt0049189,tt0054452,tt0057345,tt0059956  
3         actor,writer,soundtrack  tt0080455,tt0072562,tt0077975,tt0078723  
4           writer,director,act

In [4]:
#one row for each comma separated cell value, inspired by:
#https://stackoverflow.com/questions/50731229/split-cell-into-multiple-rows-in-pandas-dataframe

# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

#Need to convert 'primaryProfession' column to string, for some reason
df_name_basics['primaryProfession'] = df_name_basics['primaryProfession'].astype(str)

# calculate lengths of splits
lens = df_name_basics['primaryProfession'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate
                        #Repeating columns
df_name_basics_flat1 = pd.DataFrame({'nconst': np.repeat(df_name_basics['nconst'], lens),
                    'primaryName': np.repeat(df_name_basics['primaryName'], lens),
                    'birthYear': np.repeat(df_name_basics['birthYear'], lens),
                    'deathYear': np.repeat(df_name_basics['deathYear'], lens),
                    'knownForTitles': np.repeat(df_name_basics['knownForTitles'], lens),
                    #Newly split columns
                    'primaryProfession': chainer(df_name_basics['primaryProfession'])})

# calculate lengths of splits
lens = df_name_basics_flat1['knownForTitles'].str.split(',').map(len)

df_name_basics_flat2 = pd.DataFrame({'nconst': np.repeat(df_name_basics_flat1['nconst'], lens),
                    'primaryName': np.repeat(df_name_basics_flat1['primaryName'], lens),
                    'birthYear': np.repeat(df_name_basics_flat1['birthYear'], lens),
                    'deathYear': np.repeat(df_name_basics_flat1['deathYear'], lens),
                    'primaryProfession': np.repeat(df_name_basics_flat1['primaryProfession'], lens),
                    #Newly split columns
                    'knownForTitles': chainer(df_name_basics_flat1['knownForTitles'])})


print(df_name_basics_flat2[:20])


      nconst    primaryName birthYear deathYear primaryProfession  \
0  nm0000001   Fred Astaire      1899      1987        soundtrack   
0  nm0000001   Fred Astaire      1899      1987        soundtrack   
0  nm0000001   Fred Astaire      1899      1987        soundtrack   
0  nm0000001   Fred Astaire      1899      1987        soundtrack   
0  nm0000001   Fred Astaire      1899      1987             actor   
0  nm0000001   Fred Astaire      1899      1987             actor   
0  nm0000001   Fred Astaire      1899      1987             actor   
0  nm0000001   Fred Astaire      1899      1987             actor   
0  nm0000001   Fred Astaire      1899      1987     miscellaneous   
0  nm0000001   Fred Astaire      1899      1987     miscellaneous   
0  nm0000001   Fred Astaire      1899      1987     miscellaneous   
0  nm0000001   Fred Astaire      1899      1987     miscellaneous   
1  nm0000002  Lauren Bacall      1924      2014           actress   
1  nm0000002  Lauren Bacall      1

In [5]:
#filter only on titles in our universe
df_name_basics_flat3 = df_name_basics_flat2.loc[df_name_basics_flat2['knownForTitles'].isin(title_ids)]

In [7]:
#Filter on actors, writers, and directors, all in their own dataframes
df_actors = df_name_basics_flat3.loc[df_name_basics_flat3['primaryProfession'].isin(['actor','actress'])]
df_directors = df_name_basics_flat3.loc[df_name_basics_flat3['primaryProfession'].isin(['director'])]
df_writers = df_name_basics_flat3.loc[df_name_basics_flat3['primaryProfession'].isin(['writer'])]

print(df_actors.head())
print(df_directors.head())
print(df_writers.head())

         nconst         primaryName birthYear deathYear primaryProfession  \
2690  nm0002694     Leland L. Jones      1963        \N             actor   
3022  nm0003028      Rodrigo Botero        \N        \N             actor   
3819  nm0003832  Ingrid Schoelderle      1952        \N           actress   
6356  nm0006379       Axel Melzener      1975        \N             actor   
7616  nm0007664        Byron Abalos        \N        \N             actor   

     knownForTitles  
2690      tt4463894  
3022      tt4761100  
3819      tt9097694  
6356      tt5293310  
7616      tt4504044  
          nconst     primaryName birthYear deathYear primaryProfession  \
2840   nm0002844   Donna Wheeler        \N        \N          director   
2911   nm0002916       Ray Brady      1961        \N          director   
8363   nm0008443  Dominique Abel        \N        \N          director   
10890  nm0011104    Julian Adams        \N        \N          director   
15722  nm0016176    Juana Macías   

In [9]:
#Rename actor columns
df_actors.columns=['actor_id','primaryName','birthYear','deathYear','primaryProfession', 'knownForTitles']
#Drop the primaryProfession and knownFor column
df_actors = df_actors.drop(['primaryProfession'], axis=1)
#Drop duplicates
df_actors = df_actors.drop_duplicates(subset=['actor_id'])
#Then output to csv
print(df_actors.head())
print("--------------")
df_actors.to_csv('app_data/actor.csv', index=False)


#Do the same for directors and writers
df_directors.columns=['director_id','primaryName','birthYear','deathYear','primaryProfession', 'knownForTitles']
df_directors = df_directors.drop(['primaryProfession'], axis=1)
#Drop duplicates
df_directors = df_directors.drop_duplicates(subset=['director_id'])
print(df_directors.head())
print("--------------")
df_directors.to_csv('app_data/director.csv', index=False)

df_writers.columns=['writer_id','primaryName','birthYear','deathYear','primaryProfession', 'knownForTitles']
df_writers = df_writers.drop(['primaryProfession'], axis=1)
#Drop duplicates
df_writers = df_writers.drop_duplicates(subset=['writer_id'])
print(df_writers.head())
df_writers.to_csv('app_data/writer.csv', index=False)

       actor_id         primaryName birthYear deathYear knownForTitles
2690  nm0002694     Leland L. Jones      1963        \N      tt4463894
3022  nm0003028      Rodrigo Botero        \N        \N      tt4761100
3819  nm0003832  Ingrid Schoelderle      1952        \N      tt9097694
6356  nm0006379       Axel Melzener      1975        \N      tt5293310
7616  nm0007664        Byron Abalos        \N        \N      tt4504044
--------------
      director_id     primaryName birthYear deathYear knownForTitles
2840    nm0002844   Donna Wheeler        \N        \N      tt9144872
2911    nm0002916       Ray Brady      1961        \N      tt8431978
8363    nm0008443  Dominique Abel        \N        \N      tt8962546
10890   nm0011104    Julian Adams        \N        \N      tt0783640
15722   nm0016176    Juana Macías      1971        \N      tt7853636
--------------
      writer_id     primaryName birthYear deathYear knownForTitles
2911  nm0002916       Ray Brady      1961        \N      tt8431

In [26]:
#Print columns headers for manual population into MySQL workbench
print(list(df_actors))
print(list(df_directors))
print(list(df_writers))

['actor_id', 'primaryName', 'birthYear', 'deathYear']
['director_id', 'primaryName', 'birthYear', 'deathYear']
['writer_id', 'primaryName', 'birthYear', 'deathYear']
